In [1]:
import pandas as pd
from model import Poly2SLS, PredPolyRidge, PolySLSRidge, LinearModel, MedianHeuristic, PvalueLog, ConfIntHSIC
import patsy
import numpy as np
from functools import partial
from statsmodels.sandbox.regression.gmm import LinearIVGMM
from statsmodels.formula.api import ols
import itertools
from joblib import Parallel, delayed
from kernel import RBFKernel, CategoryKernel
from torch.utils.data import TensorDataset
from linearmodels.iv.model import IV2SLS
import torch
from utils import *
import warnings
warnings.simplefilter("ignore")

In [2]:
df = pd.read_csv("card.csv", index_col=0)
# binary vars
df['exp_bin'] = pd.cut(df['exper'], 3)

In [3]:
OLS = ols("lwage ~ -1 + educ + black + smsa + C(exp_bin) + south", df).fit()
formula = 'lwage ~ -1 + black + smsa + C(exp_bin) + south + [educ ~ nearc4]'
IV = IV2SLS.from_formula(formula, df).fit()

In [4]:
print(OLS.params['educ'])
print(IV.params['educ'])

0.055783902421048956
0.16743995201250073


In [5]:
OLS.conf_int()

0         1
C(exp_bin)[Interval(-0.023, 7.667, closed='righ...  5.269457  5.471592
C(exp_bin)[Interval(7.667, 15.333, closed='righ...  5.494621  5.672034
C(exp_bin)[Interval(15.333, 23.0, closed='right')]  5.592460  5.760238
educ                                                0.049303  0.062265
black                                              -0.234581 -0.163543
smsa                                                0.134997  0.197834
south                                              -0.158077 -0.097132

In [6]:
IV.conf_int()

lower     upper
C(exp_bin)[T.(-0.023, 7.667]]  1.726769  5.746643
C(exp_bin)[T.(7.667, 15.333]]  2.419886  5.909882
C(exp_bin)[T.(15.333, 23.0]]   3.257057  5.928358
black                         -0.230400  0.076805
smsa                           0.017566  0.189623
south                         -0.148241 -0.017957
educ                           0.029991  0.304889

In [7]:
def invert_test(p, X, W, Y, Z, kernels, method='gamma'):
    Y_hat = Y - p * X
    se_callback = MedianHeuristic()
    pval_callback = PvalueLog()
    
    batch_size = 128
    kernel_e = RBFKernel(sigma=1)
    kernel_z = CategoryKernel(one_hot=False)
    hsic_net = LinearModel(input_dim=W.shape[1],
                           lr=1e-2,
                           lmd=0.0,
                           kernel_e=kernel_e,
                           kernel_z=kernel_z,
                           bias=False)
    
    trainloader = torch.utils.data.DataLoader(TensorDataset(to_torch(W), to_torch(Y_hat), to_torch(Z)),
                                              batch_size=batch_size,
                                              shuffle=True, num_workers=0)
    
    max_epoch = 10
    hsic_net = fit_restart(trainloader, hsic_net, pval_callback, max_epoch, se_callback, num_restart=1)
    
    res = Y_hat - hsic_net(to_torch(W)).detach().numpy()
    res = dhsic_test(res, Z, kernels, 
                     statistics=True, method=method)
    return res

def worker_init():
    from rpy2.robjects import numpy2ri
    numpy2ri.activate()
    global dHSIC
    dHSIC = packages.importr("dHSIC")
    
def applyParallel(X, W, Y, Z, kernels, invert_test, param_range):
    func = partial(invert_test, X=X,W=W, Y=Y,Z=Z, kernels=kernels)
    from multiprocessing import Pool, cpu_count
    with Pool(cpu_count(), worker_init) as p:
        ret_list = p.map(func, param_range)
    return ret_list

In [8]:
W = np.asarray(patsy.dmatrix("~ -1 + black + smsa + C(exp_bin) + south", data=df, return_type='matrix'))
Z = np.asarray(patsy.dmatrix("~ -1 + nearc4 + black + smsa + C(exp_bin) + south", data=df, return_type='matrix'))
X = df['educ']
Y = df['lwage']

In [9]:
param_range = np.linspace(0, 0.4, 200)
kernels = ['gaussian', 'discrete']

res = applyParallel(X, W, Y, Z, kernels, invert_test, param_range)

GPU available: False, used: False
GPU available: False, used: False
GPU available: False, used: False
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
IPU available: False, using: 0 IPUs
GPU available: False, used: False
GPU available: False, used: False
IPU available: False, using: 0 IPUs
TPU available: False, using: 0 TPU cores
GPU available: False, used: False
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
GPU available: False, used: False
IPU available: False, using: 0 IPUs
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
GP

p-value: 0.0027756944199104697
iteration 1, reject the test!
p-value: 0.016557722835907916
iteration 1, reject the test!
p-value: 0.00013601871892332068p-value: 0.008780545389916568

iteration 1, reject the test!
iteration 1, reject the test!
p-value: 6.375012433014616e-05
iteration 1, reject the test!p-value: 0.0005951943154615165
p-value: 0.015684666421411503

p-value: 0.00028641686020723195p-value: 0.0008238756132915458p-value: 0.0043480136353288845
iteration 1, reject the test!

iteration 1, reject the test!iteration 1, reject the test!

iteration 1, reject the test!p-value: 0.03174122950659831
iteration 1, reject the test!


p-value: 0.005781587429440657iteration 1, reject the test!
iteration 1, reject the test!
p-value: 0.01317679925761535

iteration 1, reject the test!
p-value: 0.025269750865305918p-value: 0.0016746620597029285

iteration 1, reject the test!iteration 1, reject the test!

p-value: 0.010848340143403634
iteration 1, reject the test!


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
GPU available: False, used: False
GPU available: False, used: False
IPU available: False, using: 0 IPUs
IPU available: False, using: 0 IPUs
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
GPU available: False, used: False
IPU available: False, using: 0 IPUs
IPU available: False, using: 0 IPUs
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
GPU available: False, used: False
TPU

p-value: 9.320711398899402e-05
iteration 1, reject the test!
p-value: 0.00036123499494296156
iteration 1, reject the test!
p-value: 0.0001442315496823008
iteration 1, reject the test!
p-value: 0.00016755145498032804
iteration 1, reject the test!
p-value: 0.007390026134118818
iteration 1, reject the test!
p-value: 0.0012228960598748473
p-value: 8.665614470849447e-05iteration 1, reject the test!

iteration 1, reject the test!
p-value: 0.004517749027100307
p-value: 0.00010977415718225756iteration 1, reject the test!

iteration 1, reject the test!
p-value: 0.0008123056603326439p-value: 0.002889879905183741

p-value: 0.0005311185939811572p-value: 0.02196305553430774iteration 1, reject the test!

iteration 1, reject the test!

iteration 1, reject the test!
iteration 1, reject the test!
p-value: 0.01311828857543809
iteration 1, reject the test!


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


...

p-value: 1.413094176938113e-20
iteration 1, reject the test!
p-value: 1.56963657615322e-18
iteration 1, reject the test!


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


p-value: 5.153937412180548e-21
iteration 1, reject the test!
p-value: 1.425043333109974e-19
iteration 1, reject the test!


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


p-value: 1.2168659318989086e-20
iteration 1, reject the test!
p-value: 1.1125191966066148e-19
iteration 1, reject the test!


In [10]:
pval_df = pd.DataFrame([(param, p, s) for (param, (p,s)) in zip(param_range, res)], columns=['param', 'pval', 'stat'])

In [11]:
point_estimate = pval_df.param[pval_df.stat == pval_df.stat.min()].iloc[0]

In [12]:
accept_df = pval_df.query('pval > 0.05')
conf_int = accept_df.param.min(), accept_df.param.max()

In [13]:
print(point_estimate)
print(conf_int)

0.18291457286432164
(0.13467336683417086, 0.2291457286432161)
